In [1]:
import os

import xarray as xr
import cfgrib
import numpy as np
xr.backends.list_engines()

/Users/limingzhou/miniforge3/envs/energy/lib/python3.9/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


{'scipy': <ScipyBackendEntrypoint>
   Open netCDF files (.nc, .nc4, .cdf and .gz) using scipy in Xarray
   Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.ScipyBackendEntrypoint.html,
 'cfgrib': <CfGribBackend>
   Open GRIB files (.grib, .grib2, .grb and .grb2) in Xarray
   Learn more at https://github.com/ecmwf/cfgrib,
 'gini': <GiniXarrayBackend>,
 'rasterio': <RasterioBackend>,
 'store': <StoreBackendEntrypoint>
   Open AbstractDataStore instances in Xarray
   Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.StoreBackendEntrypoint.html}

In [3]:
import cfgrib

hrrr_obs_file = '/Users/limingzhou/zhoul/work/energy/grib2/hrrrdata/hrrrsub_2020_01_05_05F0.grib2'

utah_file = '/Users/limingzhou/zhoul/work/energy/grib2/utah/20200105.hrrr.t15z.wrfsfcf00.grib2'

data_file = hrrr_obs_file

backend_kwargs={'filter_by_keys': {'typeOfLevel': 'heightAboveGround', 'level': 10}}
ds_10m = xr.open_dataset(data_file, engine='cfgrib', backend_kwargs=backend_kwargs)

backend_kwargs['filter_by_keys']['level'] = 2
ds_2m = xr.open_dataset(data_file, engine='cfgrib', backend_kwargs=backend_kwargs)

backend_kwargs['filter_by_keys'] = {'stepType': 'instant',
                                    'typeOfLevel': 'surface'}
ds_surface = xr.open_dataset(data_file, engine='cfgrib', backend_kwargs=backend_kwargs)


In [4]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
lon1 = -97.5
lat1 = 38.5
slat = 38.5
projData= ccrs.LambertConformal(central_longitude=lon1,
                             central_latitude=lat1,
                             standard_parallels=[slat,slat],globe=ccrs.Globe(semimajor_axis=6371229,
                                        semiminor_axis=6371229))



In [5]:
d=xr.Dataset()
d['a']=[1,2,4]
print(d.keys())

KeysView(<xarray.Dataset> Size: 24B
Dimensions:  (a: 3)
Coordinates:
  * a        (a) int64 24B 1 2 4
Data variables:
    *empty*)


In [6]:
ds_10m.coords['longitude'].data[0,:] - 360

array([-122.719528  , -122.69286132, -122.6661903 , ...,  -72.3430592 ,
        -72.31638668,  -72.28971849])

In [7]:
d={'a':1, 'b':2}
d.keys()


dict_keys(['a', 'b'])

In [8]:
ds_surface

<xarray.Dataset> Size: 107MB
Dimensions:     (y: 1059, x: 1799)
Coordinates:
    time        datetime64[ns] 8B ...
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
    latitude    (y, x) float64 15MB ...
    longitude   (y, x) float64 15MB ...
    valid_time  datetime64[ns] 8B ...
Dimensions without coordinates: y, x
Data variables:
    vis         (y, x) float32 8MB ...
    gust        (y, x) float32 8MB ...
    sp          (y, x) float32 8MB ...
    t           (y, x) float32 8MB ...
    snowc       (y, x) float32 8MB ...
    sde         (y, x) float32 8MB ...
    dswrf       (y, x) float32 8MB ...
    vbdsf       (y, x) float32 8MB ...
    vddsf       (y, x) float32 8MB ...
    blh         (y, x) float32 8MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-09-14T08:15 GRIB to CDM+CF via cfgrib-0.9.1...

In [9]:
import cartopy.crs as ccrs
projection = ccrs.LambertConformal(central_longitude=262.5, 
                                   central_latitude=38.5, 
                                   standard_parallels=(38.5, 38.5),
                                    globe=ccrs.Globe(semimajor_axis=6371229,
                                                     semiminor_axis=6371229))

proj_latlon = ccrs.PlateCarree()
p=projection.transform_point(-73+360,43, proj_latlon)
p_arr = projection.transform_points(src_crs=proj_latlon, x=np.array([[-73,-72],[-71,-70]]),y=np.array([[43,42],[43,42]]))

In [10]:
p_arr[:,:,0]
    


array([[1975255.80772422, 2084338.85508273],
       [2132195.86893123, 2243107.39757602]])

In [11]:
p_arr[:,:,1]

array([[765379.66714325, 680041.79635187],
       [810008.59841197, 727061.04735771]])

In [12]:
p

(1975255.8077243313, 765379.6671432788)

In [13]:


def grb_to_grid(grb_obj):
    """Takes a single grb object containing multiple
    levels. Assumes same time, pressure levels. Compiles to a cube"""
    n_levels = len(grb_obj)
    levels = np.array([grb_element['level'] for grb_element in grb_obj])
    indexes = np.argsort(levels)[::-1] # highest pressure first
    cube = np.zeros([n_levels, grb_obj[0].values.shape[0], grb_obj[1].values.shape[1]])
    for i in range(n_levels):
        cube[i,:,:] = grb_obj[indexes[i]].values
    cube_dict = {'data' : cube, 'units' : grb_obj[0]['units'],
                 'levels' : levels[indexes]}
    return cube_dict
        
    
     


### Test Herbie

In [14]:
from herbie import Herbie


In [15]:
# Herbie object for the HRRR model 6-hr surface forecast product
H = Herbie(
  '2021-01-01 12:00',
  model='hrrr',
  product='sfc',
  fxx=6, save_dir='~/zhoul/work/herbie-data',
)

# Look at file contents
H.inventory()

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2021-Jan-01 12:00 UTC F06 ┊ GRIB2 @ aws ┊ IDX @ aws


,grib_message,start_byte,end_byte,range,reference_time,valid_time,variable,level,forecast_time,search_this
0,1,0,577438.0,0-577438,2021-01-01 12:00:00,2021-01-01 18:00:00,REFC,entire atmosphere,6 hour fcst,:REFC:entire atmosphere:6 hour fcst
1,2,577439,927042.0,577439-927042,2021-01-01 12:00:00,2021-01-01 18:00:00,RETOP,cloud top,6 hour fcst,:RETOP:cloud top:6 hour fcst
2,3,927043,1617864.0,927043-1617864,2021-01-01 12:00:00,2021-01-01 18:00:00,var discipline=0 center=7 local_table=1 parmca...,entire atmosphere,6 hour fcst,:var discipline=0 center=7 local_table=1 parmc...
3,4,1617865,2030201.0,1617865-2030201,2021-01-01 12:00:00,2021-01-01 18:00:00,VIL,entire atmosphere,6 hour fcst,:VIL:entire atmosphere:6 hour fcst
4,5,2030202,3465165.0,2030202-3465165,2021-01-01 12:00:00,2021-01-01 18:00:00,VIS,surface,6 hour fcst,:VIS:surface:6 hour fcst
...,...,...,...,...,...,...,...,...,...,...
168,169,157190383,157205277.0,157190383-157205277,2021-01-01 12:00:00,2021-01-01 18:00:00,ICEC,surface,6 hour fcst,:ICEC:surface:6 hour fcst
169,170,157205278,158809901.0,157205278-158809901,2021-01-01 12:00:00,2021-01-01 18:00:00,SBT123,top of atmosphere,6 hour fcst,:SBT123:top of atmosphere:6 hour fcst
170,171,158809902,161166255.0,158809902-161166255,2021-01-01 12:00:00,2021-01-01 18:00:00,SBT124,top of atmosphere,6 hour fcst,:SBT124:top of atmosphere:6 hour fcst
171,172,161166256,162663011.0,161166256-162663011,2021-01-01 12:00:00,2021-01-01 18:00:00,SBT113,top of atmosphere,6 hour fcst,:SBT113:top of atmosphere:6 hour fcst


### Get the mina and max datetime

In [22]:
s1=':TMP:2 m'
s_2m=':TMP:2 m|SPFH:2 m|DPT:2 m|RH:2 m'
s_surface=':VIS:surface|GUST:surface|PRES:surface|TMP:surface|SNOWC:surface|DSWRF:surface|VBDSF:surface|VDDSF:surface|HPBL:surface'
s_10m=':UGRD:10 m|VGRD:10 m|WIND:10 m'
H.inventory(search=s_surface)
#H.download(s1,verbose=True,overwrite=True)

,grib_message,start_byte,end_byte,range,reference_time,valid_time,variable,level,forecast_time,search_this
4,5,2030202,3465165.0,2030202-3465165,2021-01-01 12:00:00,2021-01-01 18:00:00,VIS,surface,6 hour fcst,:VIS:surface:6 hour fcst
8,9,4328373,5569537.0,4328373-5569537,2021-01-01 12:00:00,2021-01-01 18:00:00,GUST,surface,6 hour fcst,:GUST:surface:6 hour fcst
61,62,39278882,40802586.0,39278882-40802586,2021-01-01 12:00:00,2021-01-01 18:00:00,PRES,surface,6 hour fcst,:PRES:surface:6 hour fcst
63,64,42961055,44243535.0,42961055-44243535,2021-01-01 12:00:00,2021-01-01 18:00:00,TMP,surface,6 hour fcst,:TMP:surface:6 hour fcst
68,69,46891948,47138899.0,46891948-47138899,2021-01-01 12:00:00,2021-01-01 18:00:00,SNOWC,surface,6 hour fcst,:SNOWC:surface:6 hour fcst
125,126,98240897,100859966.0,98240897-100859966,2021-01-01 12:00:00,2021-01-01 18:00:00,DSWRF,surface,6 hour fcst,:DSWRF:surface:6 hour fcst
130,131,106725261,108385986.0,106725261-108385986,2021-01-01 12:00:00,2021-01-01 18:00:00,VBDSF,surface,6 hour fcst,:VBDSF:surface:6 hour fcst
131,132,108385987,111062831.0,108385987-111062831,2021-01-01 12:00:00,2021-01-01 18:00:00,VDDSF,surface,6 hour fcst,:VDDSF:surface:6 hour fcst
152,153,137695551,140563866.0,137695551-140563866,2021-01-01 12:00:00,2021-01-01 18:00:00,HPBL,surface,6 hour fcst,:HPBL:surface:6 hour fcst


### Get the missing hours

### Extract a rectangle, based on the four corners